In [ ]:
# let's calculate 

In [2]:
from pathlib import Path
from climada.entity import Exposures
import numpy as np
exposures_dict = {}
path_exposures = Path("/nfs/n2o/wcr/szelie/worldpop/climada_exposures")

exposures_dict = {}
for year in np.arange(2003, 2023):
    year = str(year)
    exposures_dict[year]= Exposures.from_hdf5(path_exposures / f"global_era5_025_compatible_{year}.hdf5")
    exposures_dict[year].gdf["impf_cs"] = 1 
    exposures_dict[year].gdf['longitude'] = exposures_dict[year].gdf['longitude'].apply(lambda x: x - 360 if x > 180 else x)
    exposures_dict[year].gdf['value'][exposures_dict[year].gdf['value']<0] = 0


In [3]:
import geopandas as gpd
from shapely.geometry import Point


gdf_countries = gpd.read_file("/nfs/n2o/wcr/szelie/lancet/admin_boundaries/Detailed_Boundary_ADM0")

geometry = [Point(xy) for xy in zip(exposures_dict[year].gdf.longitude, exposures_dict[year].gdf.latitude)]

gdf = gpd.GeoDataFrame(exposures_dict[year].gdf, crs="EPSG:4326", geometry=geometry)

gdf = gpd.sjoin(gdf, gdf_countries, how="left", predicate='within')

In [4]:
total_pop_by_year = []
for year in np.arange(2003, 2023):
    exposures_dict[str(year)].gdf["ISO_3_CODE"] = gdf["ISO_3_CODE"]
    gdf_year = exposures_dict[str(year)].gdf[['value','ISO_3_CODE']].groupby('ISO_3_CODE').sum()
    gdf_year['year'] = year
    total_pop_by_year.append(gdf_year)


In [5]:
import pandas as pd
pop_by_country_by_year = pd.concat(total_pop_by_year).reset_index()

In [6]:
pop_by_country_by_year.to_csv("/nfs/n2o/wcr/szelie/health_multi_risk_data/population/worldpop_by_country_by_year_2000_2022.csv")